In [1]:
import numpy as np
from pathlib import Path
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import obspy
import bisect
from pyproj import Geod
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import sys
sys.path.append('/home/Public/SiameseEQTransformer/SiameseEQT/')
from src.S_EqT_model import S_EqT_Model_create
from src.S_EqT_model import S_EqT_Model_seprate
from src.S_EqT_model import S_EqT_RSRN_Model
from src.S_EqT_model import S_EqT_HED_Model
from src.S_EqT_concate_fix_corr import S_EqT_Concate_RSRN_Model
from src.misc import get_train_list
import keras.backend
import yaml
from random import shuffle

Using TensorFlow backend.
/root/anaconda3/envs/eqt/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/root/anaconda3/envs/eqt/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/root/anaconda3/envs/eqt/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/root/anaconda3/envs/eqt/lib/python3.7/site-packag

# Load model

In [2]:
keras.backend.set_floatx('float32')
# load configuration files
cfgs = yaml.load(open('P_Test_example.yaml','r'),Loader=yaml.BaseLoader)
if_encoder_concate = int(cfgs['Model']['Encoder_concate'])
if if_encoder_concate == 1:
    encode_model, siamese_model, EqT_model = S_EqT_Concate_RSRN_Model(cfgs)
    encoder_encoded_list = cfgs['Model']['Encoder_concate_list']
    encoder_encoded_lengths = cfgs['Model']['Encoder_concate_lengths']
    encoder_encoded_channels = cfgs['Model']['Encoder_concate_channels']
        
elif int(cfgs['Model']['MODEL_RSRN'] == 0):
    encode_model, siamese_model, EqT_model = S_EqT_HED_Model(cfgs)
else:
    encode_model, siamese_model, EqT_model = S_EqT_RSRN_Model(cfgs)

if int(cfgs['Model']['LoadPretrainedModel']) == 1:
    siamese_model.load_weights(cfgs['Model']['PretrainModelPath'])

Start loading EqT model...


Start building EqT encoder model...
Start building Siamese EqT model...
Start building correlation layers...
Start sideoutput & residual layers...
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 6000, 3)      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 6000, 8)      272         input[0][0]                      
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, 3000, 8)      0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_2 (Conv1

In [3]:
siamese_model.load_weights('/home/Public/SiameseEQTransformer/SiameseEQT/models/P_Add_Noise_1023_UNI_NEW_model195000.hdf5')

# Set params

In [27]:
# For P or S
# difference is velocity and trained model and encoded model
# set up model
base_time = obspy.UTCDateTime(2010,1,1)
# search params
# search distance in km
max_search_distance = 10
# template P threshold 0.5
template_P_min_threshold = 0.5
template_S_min_threshold = 0.5

VpRefMin = 3.5
VsRefMin = 2.5

P_boundary_time = 1.0
S_boundary_time = 1.0

sta_list_file_path = '/mnt/BYEB/LA_JAN/LA_DATA/PostProcess/LAN_REAL_Use.dat'
prev_file_str = '/mnt/BYEB/LA_JAN/LA_DATA/PostProcess/EQTRes/20100101/'
search_hdf5_prefix =  '/mnt/BYEB/LA_JAN/LA_DATA/LA_2010_Jan_processed_hdfs/'
detections_prefix = '/mnt/BYEB/LA_JAN/LA_DATA/LA_2010_Jan_detections/'

In [28]:
GEOD = Geod(ellps='WGS84')
# Rough meters/degree calculation
M_PER_DEG = (GEOD.inv(0, 0, 0, 1)[2] + GEOD.inv(0, 0, 1, 0)[2]) / 2

In [29]:
def get_search_station_list(station, station_list, max_distance):
    search_list = list()
    for t_sta in station_list:
        # skip self
        if t_sta[0] == station[0]:
            continue
        # calculate distance
        t_dis = GEOD.inv(t_sta[3],t_sta[2],station[3],station[2])[2]/1000.0
        if t_dis > max_distance:
            continue
        search_list.append(t_sta)
    return search_list

In [30]:
station_list = list()
s_eqt_dict = dict()
station_list_file = open(sta_list_file_path,'r')
sta_id = 0

for line in station_list_file.readlines():
    if len(line) < 3:
        continue
    splits = line.split(' ')
    sta_name = splits[2]+'.'+splits[3]
    
    s_eqt_dict[sta_name] = dict()
    s_eqt_dict[sta_name]['P'] = list()
    s_eqt_dict[sta_name]['S'] = list()
    s_eqt_dict[sta_name]['P_Prob'] = list()
    s_eqt_dict[sta_name]['S_Prob'] = list()
    
    sta_lat = float(splits[1])
    sta_lon = float(splits[0])
    
    station_list.append( (sta_id, sta_name, sta_lat, sta_lon) )
    sta_id += 1

In [31]:
keep_t = None
pick_num_list = list()
sta_name_list = list()

for sta_key in s_eqt_dict.keys():
    S_EqT_P_list = list()
    #print(sta_key)

    s_times = list()
    s_probs = list()
    
    prev_file = prev_file_str + '{}.P.txt'.format(sta_key)
    if os.path.exists(prev_file):
        f = open(prev_file,'r')
        for line in f.readlines():
            if len(line) > 3:
                s_times.append(float(line.split(' ')[0]))
                s_probs.append(float(line.split(' ')[1]))
        f.close()
    else:
        print(prev_file)
    #print(indexs)
    #s_times = s_times[indexs]
    #s_probs = s_probs[indexs]
    s_eqt_dict[sta_key]['P'] = s_times
    s_eqt_dict[sta_key]['P_Prob'] = s_probs
    

In [32]:
train_E_num = int(cfgs['Train']['Train_E_num'])
train_set_path = cfgs['Train']['Train_data_path']
step_num = 0

print_interval = int(cfgs['Train']['Train_print_interval'])
save_interval = int(cfgs['Train']['Model_save_interval'])
save_path = cfgs['Train']['Model_save_path']

RSRN_lengths = cfgs['Model']['RSRN_Encoded_lengths']
RSRN_channels = cfgs['Model']['RSRN_Encoded_channels']

In [43]:
s_eqt_dict['CI.RIN']['P']

[86403.9848,
 135367.9948,
 136025.9448,
 141337.0548,
 197375.9948,
 407706.0048,
 500257.8748,
 650106.4048,
 653918.0348,
 679118.0348,
 729007.0448,
 748747.0348,
 922836.0248,
 922836.0548,
 999527.5448,
 1077547.0748,
 1096716.4748,
 1118827.9948,
 1136884.0948,
 1198355.9948,
 1198356.0148,
 1198356.0148,
 1198357.0748]

In [42]:
print(len(s_eqt_dict['CI.RIN']['P']))

23


In [40]:
print(len(s_eqt_dict['CI.RIN']['P']))

23


# START SEQT

In [44]:
plot_dx = 0
sta_search_dx = 0
plot_num = 0
for sta in station_list:
    sta_search_dx += 1
    print('On {} of {}'.format(sta_search_dx, len(station_list)))
    # select stations to search
    h5py_base_dir = search_hdf5_prefix
    search_list = get_search_station_list(sta, station_list, max_search_distance)
    seed_csv = detections_prefix + '{}_outputs/X_prediction_results.csv'.format(sta[1][3:],sta[1][3:])
    seed_csv_file = pd.read_csv(seed_csv)
    for e_time in seed_csv_file['file_name']:
        mask = (seed_csv_file['file_name'] == e_time)
        try:
            spt_t = obspy.UTCDateTime(seed_csv_file[mask]['p_arrival_time'].values[0]) - obspy.UTCDateTime(e_time[-27:])
            sst_t = obspy.UTCDateTime(seed_csv_file[mask]['s_arrival_time'].values[0]) - obspy.UTCDateTime(e_time[-27:])
            spt_prob = seed_csv_file[mask]['p_probability'].values[0]
            sst_prob = seed_csv_file[mask]['s_probability'].values[0]
        except:
            continue
        if spt_prob < 0.3:
            continue
        spt = np.zeros([1,1])
        spt[0,0] = float(spt_t/60.0)
        sst = np.zeros([1,1])
        sst[0,0] = float(sst_t/60.0)
        coda_end = np.zeros([1,1])
        coda_end[0,0] = float(sst_t/60.0)
        print(e_time)
        ref_pick_time = obspy.UTCDateTime(e_time[-27:]) + spt[0,0] * 60 - base_time
        print('REF TIME:{}'.format(ref_pick_time))
        seed_h5file = h5py_base_dir + '{}.hdf5'.format(sta[1][3:])
        with h5py.File(seed_h5file,'r') as f:
            dataset = f.get('data/'+e_time)
            data_t = np.array(dataset)
            data_t -= np.mean(data_t, axis=0 ,keepdims=True)
            t_std = np.std(data_t, axis = 0, keepdims=True)
            t_std[t_std == 0] = 1.0
            data_t /= t_std
            data_t -= np.mean(data_t, axis=0 ,keepdims=True)
            data_t_in = np.zeros([1,6000,3])
            data_t_in[0,:,:] = data_t
            
        for search_sta in search_list:
            search_ref_picks = s_eqt_dict[search_sta[1]]['P'] 
            if len(search_ref_picks) > 1:
                ref_key_id = bisect.bisect_left(search_ref_picks,ref_pick_time)
                try:
                    ref_left = np.abs(search_ref_picks[ref_key_id-1] - ref_pick_time)
                    ref_middle = np.abs(search_ref_picks[ref_key_id] - ref_pick_time)
                    ref_right = np.abs(search_ref_picks[ref_key_id+1] - ref_pick_time)
                    if ref_left < 60.0 or ref_middle < 60.0 or ref_right < 60.0:
                        print('Skipping {}'.format(search_sta[1]))
                        continue
                except:
                    pass
            else:
                pass
            h5py_base_dir_search = search_hdf5_prefix
            search_csvfile = h5py_base_dir_search + '{}.csv'.format(search_sta[1][3:])
            search_csvfile = pd.read_csv(search_csvfile)
            keys = list(search_csvfile['trace_name'])
            if len(keys) < 2:
                continue
            prefix = keys[0][:-27]
            keys = [key[-27:] for key in keys]
            key_id = bisect.bisect_left(keys,e_time[-27:])
            
            try:
                if obspy.UTCDateTime(keys[key_id]) - obspy.UTCDateTime(e_time[-27:]) < 10:
                    search_keys = [keys[key_id-1],keys[key_id],keys[key_id+1]]
                else:
                    search_keys = [keys[key_id-1],keys[key_id],keys[key_id-2]]
            except:
                continue
            # calculate key search range
            
            search_h5file =  h5py_base_dir_search + '{}.hdf5'.format(search_sta[1][3:])
            t_update_list = list()
            t_update_list_prob = list()
            with h5py.File(search_h5file,'r') as f:
                max_pred_amp = 0
                for search_key in search_keys:
                    t_search_key = prefix + search_key
                    dataset = f.get('data/'+t_search_key)
                    data_s = np.array(dataset)
                    data_s -= np.mean(data_s, axis=0 ,keepdims=True)
                    t_std = np.std(data_s, axis = 0, keepdims=True)
                    t_std[t_std == 0] = 1.0
                    data_s /= t_std

                    data_s_in = np.zeros([1,6000,3])
                    data_s_in[0,:,:] = data_s

                    encoded_t = encode_model.predict(data_t_in)
                    encoded_s = encode_model.predict(data_s_in)

                    siamese_input_list = list()

                    for rdx in range(len(RSRN_lengths)):
                        temp_length = float(RSRN_lengths[rdx])
                        template_s = int(spt[0,0]*temp_length) - 1
                        template_e = int(coda_end[0,0]*temp_length) + 1
                        template_w = int(template_e - template_s)
                        encoded_t[rdx] = encoded_t[rdx][:,template_s:template_e,:]/float(template_w)
                        encoded_t[rdx] = encoded_t[rdx].reshape([1,template_w,1,int(RSRN_channels[rdx])])
                        encoded_s[rdx] = encoded_s[rdx].reshape([1,int(RSRN_lengths[rdx]),1,int(RSRN_channels[rdx])])

                        # channel normalization
                        for channel_dx in range(int(RSRN_channels[rdx])):
                            encoded_s[rdx][0,:,0,channel_dx] -= np.max(encoded_s[rdx][0,:,0,channel_dx])
                            half_window_len = int( 200.0*temp_length/6000.0   ) + 1
                            #window_mean = np.mean(encoded_s[rdx][0,half_window_len:-half_window_len,0,channel_dx])

                            encoded_s[rdx][0,:half_window_len,0,channel_dx] = encoded_s[rdx][0,half_window_len,0,channel_dx]
                            encoded_s[rdx][0,-half_window_len:,0,channel_dx] =  encoded_s[rdx][0,-half_window_len,0,channel_dx]

                            encoded_s[rdx][0,:,0,channel_dx] *= -1.0
                            encoded_s[rdx][0,:,0,channel_dx] -= np.mean(encoded_s[rdx][0,:,0,channel_dx])
                            t_max = np.max(np.abs(encoded_s[rdx][0,:,0,channel_dx]))
                            if t_max < 0.01:
                                t_max = 1
                            encoded_s[rdx][0,:,0,channel_dx] /= t_max

                            encoded_t[rdx][0,:,0,channel_dx] -= np.max(encoded_t[rdx][0,:,0,channel_dx])
                            encoded_t[rdx][0,:,0,channel_dx] *= -1.0
                            encoded_t[rdx][0,:,0,channel_dx] -= np.mean(encoded_t[rdx][0,:,0,channel_dx])
                            t_max = np.max(np.abs(encoded_t[rdx][0,:,0,channel_dx]))
                            if t_max < 0.01:
                                t_max = 1
                            encoded_t[rdx][0,:,0,channel_dx] /= t_max

                        siamese_input_list.append(encoded_t[rdx])
                        siamese_input_list.append(encoded_s[rdx])

                    #print('RSRN Channel_normal OK')
                    if if_encoder_concate == 1:
                        for rdx in range(len(RSRN_lengths), len(RSRN_lengths) + len(encoder_encoded_list)):
                            rdx_2 = rdx - len(RSRN_lengths) 
                            temp_length = float(encoder_encoded_lengths[rdx_2])
                            template_s = int(spt[0,0]*temp_length) - 1
                            template_e = int(coda_end[0,0]*temp_length) + 1
                            template_w = int(template_e - template_s)
                            #print('Concate 1 OK')
                            encoded_t[rdx] = encoded_t[rdx][:,template_s:template_e,:]/float(template_w)
                            encoded_t[rdx] = encoded_t[rdx].reshape([1,template_w,1,int(encoder_encoded_channels[rdx_2])])
                            encoded_s[rdx] = encoded_s[rdx].reshape([1,int(encoder_encoded_lengths[rdx_2]),1,int(encoder_encoded_channels[rdx_2])])
                            #print('Concate 2 OK')
                            # channel normalization
                            for channel_dx in range(int(encoder_encoded_channels[rdx_2])):
                                encoded_s[rdx][0,:,0,channel_dx] -= np.max(encoded_s[rdx][0,:,0,channel_dx])
                                half_window_len = int( 200.0*temp_length/6000.0   ) + 1
                                #window_mean = np.mean(encoded_s[rdx][0,half_window_len:-half_window_len,0,channel_dx])
                                #print('Concate 3 OK')
                                encoded_s[rdx][0,:half_window_len,0,channel_dx] = encoded_s[rdx][0,half_window_len,0,channel_dx]
                                encoded_s[rdx][0,-half_window_len:,0,channel_dx] =  encoded_s[rdx][0,-half_window_len,0,channel_dx]
                                #print('Concate 4 OK')
                                encoded_s[rdx][0,:,0,channel_dx] *= -1.0
                                encoded_s[rdx][0,:,0,channel_dx] -= np.mean(encoded_s[rdx][0,:,0,channel_dx])
                                t_max = np.max(np.abs(encoded_s[rdx][0,:,0,channel_dx]))
                                if t_max < 0.01:
                                    t_max = 1
                                encoded_s[rdx][0,:,0,channel_dx] /= t_max
                                #print('Concate 5 OK')
                                encoded_t[rdx][0,:,0,channel_dx] -= np.max(encoded_t[rdx][0,:,0,channel_dx])
                                encoded_t[rdx][0,:,0,channel_dx] *= -1.0
                                encoded_t[rdx][0,:,0,channel_dx] -= np.mean(encoded_t[rdx][0,:,0,channel_dx])
                                t_max = np.max(np.abs(encoded_t[rdx][0,:,0,channel_dx]))
                                if t_max < 0.01:
                                    t_max = 1
                                encoded_t[rdx][0,:,0,channel_dx] /= t_max
                                #print('Concate 6 OK')
                            siamese_input_list.append(encoded_t[rdx])
                            siamese_input_list.append(encoded_s[rdx])
                    pred_res = siamese_model.predict(siamese_input_list)
                    siamese_s = np.argmax(pred_res[-1][0,:,0,0])
                    pred_amp = pred_res[-1][0,siamese_s,0,0]
                    
                    if pred_amp > 0.1 and siamese_s > 200 and siamese_s < 5800:
                        siamese_s_time = obspy.UTCDateTime(search_key) + siamese_s * 0.01
                        t_update_time  = siamese_s_time - base_time
                        t_diff_time = siamese_s_time - (obspy.UTCDateTime(e_time[-27:]) + spt[0,0] * 60)
                        #r_snr = np.max(np.abs(data_s_in[0,siamese_s-50:siamese_s,0]))/np.max(np.abs(data_s_in[0,siamese_s:siamese_s+100,0]))
                        
                        if np.abs(t_diff_time) < 6:
                            pass
                        else:
                            continue
                        t_update_list.append(t_update_time)
                        t_update_list_prob.append(pred_amp)
                        
                if len(t_update_list) > 0:
                    max_arg = np.argmax(t_update_list_prob)
                    t_update_time = t_update_list[max_arg]
                    bisect.insort(s_eqt_dict[search_sta[1]]['P'], t_update_time)
                    print('{}'.format(t_update_time))
    

On 1 of 41
ALP_CI_HH_2010-01-01T02:55:30.002700Z
REF TIME:10534.1027
Skipping CI.LEO
ALP_CI_HH_2010-01-02T03:16:00.002700Z
REF TIME:98166.9727
Skipping CI.LEO
ALP_CI_HH_2010-01-08T21:59:00.002700Z
REF TIME:683944.0327
Skipping CI.LEO
ALP_CI_HH_2010-01-11T16:38:30.002700Z
REF TIME:923915.8327
Skipping CI.LEO
ALP_CI_HH_2010-01-11T23:34:00.002700Z
REF TIME:948857.8227
Skipping CI.LEO
ALP_CI_HH_2010-01-12T02:36:30.002700Z
REF TIME:959794.0027
Skipping CI.LEO
ALP_CI_HH_2010-01-13T20:31:30.002700Z
REF TIME:1110695.9927
ALP_CI_HH_2010-01-14T19:07:00.002700Z
REF TIME:1192027.9827
Skipping CI.LEO
ALP_CI_HH_2010-01-14T19:07:00.002700Z
REF TIME:1192027.9827
Skipping CI.LEO
ALP_CI_HH_2010-01-14T21:34:30.002700Z
REF TIME:1200901.5627
Skipping CI.LEO
ALP_CI_HH_2010-01-14T21:48:00.002700Z
REF TIME:1201689.0027
1201685.99
On 2 of 41
VCS_CI_HH_2010-01-01T02:55:30.000700Z
REF TIME:10535.0507
VCS_CI_HH_2010-01-02T03:15:30.000700Z
REF TIME:98164.7507
VCS_CI_HH_2010-01-04T22:37:00.000700Z
REF TIME:340657.2

RHC_CI_EH_2010-01-02T04:04:30.009900Z
REF TIME:101090.8499
RHC_CI_EH_2010-01-04T01:22:30.000000Z
REF TIME:264156.01
RHC_CI_EH_2010-01-04T20:08:30.000000Z
REF TIME:331714.23
Skipping CI.RUS
RHC_CI_EH_2010-01-05T15:17:30.000000Z
REF TIME:400692.22
Skipping CI.RUS
RHC_CI_EH_2010-01-11T23:34:00.000000Z
REF TIME:948850.92
Skipping CI.RUS
RHC_CI_EH_2010-01-14T00:00:00.000000Z
REF TIME:1123204.98
1123204.0288
On 11 of 41
VVD_CI_EH_2010-01-04T00:36:00.001300Z
REF TIME:261362.6513
VVD_CI_EH_2010-01-05T04:48:30.000200Z
REF TIME:362936.2202
VVD_CI_EH_2010-01-05T04:49:30.000200Z
REF TIME:363011.4102
Skipping CI.PDE
VVD_CI_EH_2010-01-05T05:14:00.000200Z
REF TIME:364474.6802
Skipping CI.PDE
VVD_CI_EH_2010-01-05T05:14:00.000200Z
REF TIME:364474.6802
Skipping CI.PDE
VVD_CI_EH_2010-01-05T07:49:00.000200Z
REF TIME:373747.1102
Skipping CI.PDE
VVD_CI_EH_2010-01-05T07:53:30.000200Z
REF TIME:374037.0802
VVD_CI_EH_2010-01-05T07:59:00.000200Z
REF TIME:374381.7402
Skipping CI.PDE
VVD_CI_EH_2010-01-07T01:40:30.

SMV_CI_EH_2010-01-05T01:54:00.000000Z
REF TIME:352486.03
SMV_CI_EH_2010-01-05T03:01:00.000000Z
REF TIME:356492.01
Skipping CI.BLC
SMV_CI_EH_2010-01-05T04:41:00.000000Z
REF TIME:362493.02
Skipping CI.BLC
SMV_CI_EH_2010-01-05T04:51:30.000000Z
REF TIME:363108.96
SMV_CI_EH_2010-01-05T05:19:00.000000Z
REF TIME:364751.02
SMV_CI_EH_2010-01-05T05:59:00.000000Z
REF TIME:367168.96
SMV_CI_EH_2010-01-05T06:22:30.000000Z
REF TIME:368597.0
Skipping CI.BLC
SMV_CI_EH_2010-01-05T08:40:30.000000Z
REF TIME:376849.31
SMV_CI_EH_2010-01-05T08:50:00.000000Z
REF TIME:377428.88
SMV_CI_EH_2010-01-05T10:52:00.000000Z
REF TIME:384737.01
SMV_CI_EH_2010-01-05T12:37:30.000000Z
REF TIME:391065.04
SMV_CI_EH_2010-01-05T15:09:30.000000Z
REF TIME:400174.0
Skipping CI.BLC
SMV_CI_EH_2010-01-05T15:14:00.000000Z
REF TIME:400482.99
Skipping CI.BLC
SMV_CI_EH_2010-01-05T17:19:30.000000Z
REF TIME:407985.52
SMV_CI_EH_2010-01-05T17:27:30.000000Z
REF TIME:408458.97
Skipping CI.BLC
SMV_CI_EH_2010-01-05T19:45:00.000000Z
REF TIME:4167

Skipping CI.LCG
SMF_CI_EH_2010-01-04T06:48:00.001300Z
REF TIME:283704.8713
Skipping CI.LCG
SMF_CI_EH_2010-01-04T14:56:30.001300Z
REF TIME:313026.5713
Skipping CI.SMS
Skipping CI.DJJ
Skipping CI.LCG
Skipping CI.DJJB
SMF_CI_EH_2010-01-04T20:08:00.001300Z
REF TIME:331707.1713
Skipping CI.SMS
Skipping CI.DJJ
Skipping CI.LCG
Skipping CI.DJJB
SMF_CI_EH_2010-01-07T17:07:00.000000Z
REF TIME:580057.39
Skipping CI.SMS
Skipping CI.DJJ
Skipping CI.LCG
Skipping CI.DJJB
SMF_CI_EH_2010-01-08T05:21:00.000000Z
REF TIME:624101.3
Skipping CI.SMS
Skipping CI.DJJ
Skipping CI.LCG
Skipping CI.DJJB
SMF_CI_EH_2010-01-08T18:50:00.000000Z
REF TIME:672635.67
Skipping CI.SMS
Skipping CI.DJJ
Skipping CI.LCG
Skipping CI.DJJB
SMF_CI_EH_2010-01-09T00:07:30.000000Z
REF TIME:691661.72
Skipping CI.SMS
Skipping CI.DJJ
Skipping CI.LCG
SMF_CI_EH_2010-01-09T02:13:00.000000Z
REF TIME:699186.43
Skipping CI.SMS
Skipping CI.DJJ
Skipping CI.LCG
Skipping CI.DJJB
SMF_CI_EH_2010-01-09T12:56:00.000000Z
REF TIME:737784.47
Skipping CI.

LCG_CI_HH_2010-01-13T00:55:30.004800Z
REF TIME:1040175.9248
Skipping CI.USC
Skipping CI.SMS
On 25 of 41
MWC_CI_HH_2010-01-01T02:55:30.000700Z
REF TIME:10539.0007
Skipping CI.KIK
MWC_CI_HH_2010-01-01T03:01:00.000700Z
REF TIME:10885.5007
MWC_CI_HH_2010-01-01T03:26:00.000700Z
REF TIME:12364.0207
Skipping CI.KIK
MWC_CI_HH_2010-01-01T14:23:30.000700Z
REF TIME:51817.9307
Skipping CI.KIK
MWC_CI_HH_2010-01-01T22:20:00.000700Z
REF TIME:80432.8307
MWC_CI_HH_2010-01-02T03:15:30.000700Z
REF TIME:98162.0307
Skipping CI.KIK
MWC_CI_HH_2010-01-02T18:11:30.000700Z
REF TIME:151912.4707
MWC_CI_HH_2010-01-03T01:32:30.000700Z
REF TIME:178354.2307
Skipping CI.KIK
MWC_CI_HH_2010-01-03T01:43:00.000700Z
REF TIME:179001.8707
MWC_CI_HH_2010-01-03T03:07:00.000700Z
REF TIME:184057.5907
Skipping CI.KIK
MWC_CI_HH_2010-01-03T03:52:30.000700Z
REF TIME:186758.1207
Skipping CI.KIK
MWC_CI_HH_2010-01-03T12:30:00.000700Z
REF TIME:217824.5707
MWC_CI_HH_2010-01-03T12:39:30.000700Z
REF TIME:218399.7807
Skipping CI.KIK
MWC_CI_

...

QAL_CI_EH_2010-01-14T03:59:30.000000Z
REF TIME:1137576.01
QAL_CI_EH_2010-01-14T04:21:30.000000Z
REF TIME:1138896.12
QAL_CI_EH_2010-01-14T04:30:30.000000Z
REF TIME:1139437.87
QAL_CI_EH_2010-01-14T04:54:30.000000Z
REF TIME:1140878.4
QAL_CI_EH_2010-01-14T04:58:00.000000Z
REF TIME:1141089.28
QAL_CI_EH_2010-01-14T19:45:30.000000Z
REF TIME:1194333.99
QAL_CI_EH_2010-01-14T20:04:30.000000Z
REF TIME:1195499.46
QAL_CI_EH_2010-01-14T22:16:00.000000Z
REF TIME:1203366.01
On 28 of 41
DJJB_CI_HN_2010-01-01T04:34:00.004600Z
REF TIME:16465.2846
Skipping CI.WNS
Skipping CI.DJJ
DJJB_CI_HN_2010-01-01T22:20:00.004600Z
REF TIME:80428.8346
Skipping CI.DJJ
DJJB_CI_HN_2010-01-02T03:15:30.004600Z
REF TIME:98156.6746
Skipping CI.WNS
Skipping CI.DJJ
DJJB_CI_HN_2010-01-04T20:08:30.004400Z
REF TIME:331713.9944
Skipping CI.WNS
Skipping CI.SMF
Skipping CI.DJJ
DJJB_CI_HN_2010-01-04T21:25:00.004400Z
REF TIME:336334.8944
Skipping CI.SMF
Skipping CI.DJJ
DJJB_CI_HN_2010-01-04T21:28:00.004400Z
REF TIME:336512.3544
Skipping

IR2_CI_EH_2010-01-12T07:54:30.001300Z
REF TIME:978894.2013
IR2_CI_EH_2010-01-13T00:56:00.000900Z
REF TIME:1040181.7409
IR2_CI_EH_2010-01-13T23:11:00.000900Z
REF TIME:1120309.3709
IR2_CI_EH_2010-01-14T08:15:00.001300Z
REF TIME:1152946.4013
IR2_CI_EH_2010-01-14T09:57:30.001300Z
REF TIME:1159066.3613
IR2_CI_EH_2010-01-14T12:10:30.001300Z
REF TIME:1167036.0013
Skipping CI.QUG
IR2_CI_EH_2010-01-14T12:57:30.001300Z
REF TIME:1169863.0313
Skipping CI.QUG
IR2_CI_EH_2010-01-14T15:29:00.001300Z
REF TIME:1178944.0013
Skipping CI.QUG
IR2_CI_EH_2010-01-14T20:22:30.001300Z
REF TIME:1196566.7013
IR2_CI_EH_2010-01-14T22:50:30.001300Z
REF TIME:1205440.7913
IR2_CI_EH_2010-01-14T22:52:30.001300Z
REF TIME:1205586.7613
Skipping CI.QUG
IR2_CI_EH_2010-01-14T23:52:30.001300Z
REF TIME:1209158.8013
On 30 of 41
RUS_CI_HH_2010-01-12T02:36:00.008800Z
REF TIME:959786.7888
Skipping CI.GVR
On 31 of 41
PASC_CI_HH_2010-01-01T02:55:30.008300Z
REF TIME:10539.9983
Skipping CI.GSA
10540.5544
Skipping CI.CBC
Skipping CI.KIK


SPF_CI_HH_2010-01-07T08:00:30.000400Z
REF TIME:547260.7004
SPF_CI_HH_2010-01-07T15:22:00.000400Z
REF TIME:573757.2204
SPF_CI_HH_2010-01-07T15:28:00.000400Z
REF TIME:574086.9604
SPF_CI_HH_2010-01-07T15:40:00.000400Z
REF TIME:574809.8004
SPF_CI_HH_2010-01-07T20:51:00.000400Z
REF TIME:593465.9904
SPF_CI_HH_2010-01-08T02:36:30.000400Z
REF TIME:614195.9604
SPF_CI_HH_2010-01-08T06:29:00.000400Z
REF TIME:628168.0604
SPF_CI_HH_2010-01-08T09:57:30.000400Z
REF TIME:640654.0004
SPF_CI_HH_2010-01-08T16:06:00.000400Z
REF TIME:662765.7504
SPF_CI_HH_2010-01-10T00:29:30.000400Z
REF TIME:779377.0004
SPF_CI_HH_2010-01-10T04:59:00.000400Z
REF TIME:795581.1504
SPF_CI_HH_2010-01-10T16:08:00.000400Z
REF TIME:835689.0004
SPF_CI_HH_2010-01-10T20:29:30.000400Z
REF TIME:851377.4604
SPF_CI_HH_2010-01-11T01:44:00.000400Z
REF TIME:870259.2604
SPF_CI_HH_2010-01-11T03:01:00.000400Z
REF TIME:874888.3004
SPF_CI_HH_2010-01-11T15:08:30.000400Z
REF TIME:918517.9804
SPF_CI_HH_2010-01-11T23:34:00.000400Z
REF TIME:948858.14

STT_CI_EH_2010-01-07T22:39:30.000000Z
REF TIME:600002.67
STT_CI_EH_2010-01-07T22:51:00.000000Z
REF TIME:600676.33
STT_CI_EH_2010-01-07T22:57:30.000000Z
REF TIME:601072.48
STT_CI_EH_2010-01-07T22:59:00.000000Z
REF TIME:601181.22
STT_CI_EH_2010-01-07T23:04:30.000000Z
REF TIME:601513.72
STT_CI_EH_2010-01-07T23:07:00.000000Z
REF TIME:601657.44
STT_CI_EH_2010-01-08T00:03:00.000000Z
REF TIME:604984.97
STT_CI_EH_2010-01-08T00:17:00.000000Z
REF TIME:605855.86
STT_CI_EH_2010-01-08T00:32:30.000000Z
REF TIME:606767.04
STT_CI_EH_2010-01-08T00:32:30.000000Z
REF TIME:606767.04
STT_CI_EH_2010-01-08T00:39:00.000000Z
REF TIME:607153.12
STT_CI_EH_2010-01-08T00:46:30.000000Z
REF TIME:607594.99
STT_CI_EH_2010-01-08T01:02:00.000000Z
REF TIME:608535.43
STT_CI_EH_2010-01-08T01:50:00.000000Z
REF TIME:611424.42
STT_CI_EH_2010-01-08T01:51:30.000000Z
REF TIME:611504.47
STT_CI_EH_2010-01-08T01:56:00.000000Z
REF TIME:611774.76
STT_CI_EH_2010-01-08T01:56:30.000000Z
REF TIME:611834.67
STT_CI_EH_2010-01-08T01:58:00.0

STT_CI_EH_2010-01-14T02:23:30.000000Z
REF TIME:1131840.37
STT_CI_EH_2010-01-14T02:45:30.000000Z
REF TIME:1133146.28
STT_CI_EH_2010-01-14T02:57:30.000000Z
REF TIME:1133874.2
STT_CI_EH_2010-01-14T03:11:30.000000Z
REF TIME:1134694.99
STT_CI_EH_2010-01-14T04:02:00.000000Z
REF TIME:1137745.49
STT_CI_EH_2010-01-14T04:03:30.000000Z
REF TIME:1137866.02
STT_CI_EH_2010-01-14T04:15:30.000000Z
REF TIME:1138586.23
STT_CI_EH_2010-01-14T05:01:00.000000Z
REF TIME:1141308.58
STT_CI_EH_2010-01-14T06:17:00.000000Z
REF TIME:1145850.21
STT_CI_EH_2010-01-14T07:35:00.000000Z
REF TIME:1150520.18
STT_CI_EH_2010-01-14T08:25:00.000000Z
REF TIME:1153521.03
STT_CI_EH_2010-01-14T08:28:30.000000Z
REF TIME:1153741.03
STT_CI_EH_2010-01-14T19:59:00.000000Z
REF TIME:1195183.22
STT_CI_EH_2010-01-14T20:01:00.000000Z
REF TIME:1195300.9
STT_CI_EH_2010-01-14T20:14:00.000000Z
REF TIME:1196071.41
STT_CI_EH_2010-01-14T20:14:30.000000Z
REF TIME:1196100.85
STT_CI_EH_2010-01-14T20:18:00.000000Z
REF TIME:1196298.53
STT_CI_EH_2010-0

In [50]:
for sta_key in s_eqt_dict.keys():
    cur_file = '/mnt/BYEB/LA_JAN/LA_DATA/PostProcess/SEQTRes/20100101/{}.P.txt'.format(sta_key)
    f = open(cur_file,'w')
    arrival_dx = 0
    pre_arrival = -1
    for arrival in s_eqt_dict[sta_key]['P']:
        if arrival_dx == 0:
            pre_arrival = arrival
            arrival_dx += 1
        else:
            if np.abs(pre_arrival - arrival) < 1:
                continue
            else:
                pre_arrival = arrival
                
        S_res = '{:.3f} 0.80 0.00000000'.format(arrival)
        f.write(S_res+'\n')
    f.close()

In [26]:
for sta_key in s_eqt_dict.keys():
    s_times = s_eqt_dict[sta_key]['P']
    savefile = open('/mnt/BYEB/LA_JAN/LA_DATA/PostProcess/SEQTRes/20100101/{}.P.txt'.format(sta_key),'w')
    for s_t in s_times:
        savefile.write('{:.3f} 0.80000 0.00000000\n'.format(s_t))
    savefile.close()

In [ ]:
np.save('seqt_KLTG_1116_P.npy',s_eqt_dict)

In [ ]:
len(s_eqt_dict)

In [ ]:
# test 10
plot_dx = 0
sta_search_dx = 0
plot_num = 0
for sta in station_list:
    sta_search_dx += 1
    print('On {} of {}'.format(sta_search_dx, len(station_list)))
    # select stations to search
    search_list = get_search_station_list(sta, station_list, max_search_distance)
    seed_csv = '/run/media/root/Backup Plus/XJ2SEQT/{}_detections/{}_outputs/X_prediction_results.csv'.format(time_label,sta[1][3:])
    seed_csv_file = pd.read_csv(seed_csv)
    for e_time in seed_csv_file['file_name']:
        mask = (seed_csv_file['file_name'] == e_time)
        try:
            spt_t = obspy.UTCDateTime(seed_csv_file[mask]['p_arrival_time'].values[0]) - obspy.UTCDateTime(e_time[-27:])
            sst_t = obspy.UTCDateTime(seed_csv_file[mask]['s_arrival_time'].values[0]) - obspy.UTCDateTime(e_time[-27:])
            spt_prob = seed_csv_file[mask]['p_probability'].values[0]
            sst_prob = seed_csv_file[mask]['s_probability'].values[0]
        except:
            continue
        if spt_prob < 0.6:
            continue
        spt = np.zeros([1,1])
        spt[0,0] = float(spt_t/60.0)
        sst = np.zeros([1,1])
        sst[0,0] = float(sst_t/60.0)
        coda_end = np.zeros([1,1])
        coda_end[0,0] = float(sst_t/60.0)
        print(e_time)
        ref_pick_time = obspy.UTCDateTime(e_time[-27:]) + spt[0,0] * 60 - obspy.UTCDateTime(2018,6,19)
        
        
        
        seed_h5file = h5py_base_dir + '{}.hdf5'.format(sta[1][3:])
        with h5py.File(seed_h5file,'r') as f:
            dataset = f.get('data/'+e_time)
            data_t = np.array(dataset)
            data_t -= np.mean(data_t, axis=0 ,keepdims=True)
            t_std = np.std(data_t, axis = 0, keepdims=True)
            t_std[t_std == 0] = 1.0
            data_t /= t_std
            data_t -= np.mean(data_t, axis=0 ,keepdims=True)
            data_t_in = np.zeros([1,6000,3])
            data_t_in[0,:,:] = data_t
            
        for search_sta in search_list:
            # check if pick exists 
            #print(search_sta)
            search_ref_picks = s_eqt_dict[search_sta[1]]['P'] 
            if len(search_ref_picks) > 1:
                ref_key_id = bisect.bisect_left(search_ref_picks,ref_pick_time)
            else:
                pass
            
            try:
                ref_left = np.abs(search_ref_picks[ref_key_id-1] - ref_pick_time)
                ref_middle = np.abs(search_ref_picks[ref_key_id] - ref_pick_time)
                ref_right = np.abs(search_ref_picks[ref_key_id+1] - ref_pick_time)
                if ref_left < 6.0 or ref_middle < 6.0 or ref_right < 6.0:
                    print('Skipping {}'.format(search_sta[1]))
                    continue
            except:
                pass
            
            search_csvfile = h5py_base_dir + '{}.csv'.format(search_sta[1][3:])
            search_csvfile = pd.read_csv(search_csvfile)
            keys = list(search_csvfile['trace_name'])
            prefix = keys[0][:-27]
            keys = [key[-27:] for key in keys]
            key_id = bisect.bisect_left(keys,e_time[-27:])
            
            try:
                search_keys = [keys[key_id-2],keys[key_id-1],keys[key_id],keys[key_id+1],keys[key_id+2]]
            except:
                continue
            # calculate key search range

            search_h5file =  h5py_base_dir + '{}.hdf5'.format(search_sta[1][3:])
            with h5py.File(search_h5file,'r') as f:
                max_pred_amp = 0
                for search_key in search_keys:
                    t_search_key = prefix + search_key
                    dataset = f.get('data/'+t_search_key)
                    data_s = np.array(dataset)
                    data_s -= np.mean(data_s, axis=0 ,keepdims=True)
                    t_std = np.std(data_s, axis = 0, keepdims=True)
                    t_std[t_std == 0] = 1.0
                    data_s /= t_std

                    data_s_in = np.zeros([1,6000,3])
                    data_s_in[0,:,:] = data_s

                    encoded_t = encode_model.predict(data_t_in)
                    encoded_s = encode_model.predict(data_s_in)

                    siamese_input_list = list()

                    for rdx in range(len(RSRN_lengths)):
                        temp_length = float(RSRN_lengths[rdx])
                        template_s = int(spt[0,0]*temp_length) - 1
                        template_e = int(coda_end[0,0]*temp_length) + 1
                        template_w = int(template_e - template_s)
                        encoded_t[rdx] = encoded_t[rdx][:,template_s:template_e,:]/float(template_w)
                        encoded_t[rdx] = encoded_t[rdx].reshape([1,template_w,1,int(RSRN_channels[rdx])])
                        encoded_s[rdx] = encoded_s[rdx].reshape([1,int(RSRN_lengths[rdx]),1,int(RSRN_channels[rdx])])

                        # channel normalization
                        for channel_dx in range(int(RSRN_channels[rdx])):
                            encoded_s[rdx][0,:,0,channel_dx] -= np.max(encoded_s[rdx][0,:,0,channel_dx])
                            half_window_len = int( 200.0*temp_length/6000.0   ) + 1
                            #window_mean = np.mean(encoded_s[rdx][0,half_window_len:-half_window_len,0,channel_dx])

                            encoded_s[rdx][0,:half_window_len,0,channel_dx] = encoded_s[rdx][0,half_window_len,0,channel_dx]
                            encoded_s[rdx][0,-half_window_len:,0,channel_dx] =  encoded_s[rdx][0,-half_window_len,0,channel_dx]

                            encoded_s[rdx][0,:,0,channel_dx] *= -1.0
                            encoded_s[rdx][0,:,0,channel_dx] -= np.mean(encoded_s[rdx][0,:,0,channel_dx])
                            t_max = np.max(np.abs(encoded_s[rdx][0,:,0,channel_dx]))
                            if t_max < 0.01:
                                t_max = 1
                            encoded_s[rdx][0,:,0,channel_dx] /= t_max

                            encoded_t[rdx][0,:,0,channel_dx] -= np.max(encoded_t[rdx][0,:,0,channel_dx])
                            encoded_t[rdx][0,:,0,channel_dx] *= -1.0
                            encoded_t[rdx][0,:,0,channel_dx] -= np.mean(encoded_t[rdx][0,:,0,channel_dx])
                            t_max = np.max(np.abs(encoded_t[rdx][0,:,0,channel_dx]))
                            if t_max < 0.01:
                                t_max = 1
                            encoded_t[rdx][0,:,0,channel_dx] /= t_max

                        siamese_input_list.append(encoded_t[rdx])
                        siamese_input_list.append(encoded_s[rdx])

                    #print('RSRN Channel_normal OK')
                    if if_encoder_concate == 1:
                        for rdx in range(len(RSRN_lengths), len(RSRN_lengths) + len(encoder_encoded_list)):
                            rdx_2 = rdx - len(RSRN_lengths) 
                            temp_length = float(encoder_encoded_lengths[rdx_2])
                            template_s = int(spt[0,0]*temp_length) - 1
                            template_e = int(coda_end[0,0]*temp_length) + 1
                            template_w = int(template_e - template_s)
                            #print('Concate 1 OK')
                            encoded_t[rdx] = encoded_t[rdx][:,template_s:template_e,:]/float(template_w)
                            encoded_t[rdx] = encoded_t[rdx].reshape([1,template_w,1,int(encoder_encoded_channels[rdx_2])])
                            encoded_s[rdx] = encoded_s[rdx].reshape([1,int(encoder_encoded_lengths[rdx_2]),1,int(encoder_encoded_channels[rdx_2])])
                            #print('Concate 2 OK')
                            # channel normalization
                            for channel_dx in range(int(encoder_encoded_channels[rdx_2])):
                                encoded_s[rdx][0,:,0,channel_dx] -= np.max(encoded_s[rdx][0,:,0,channel_dx])
                                half_window_len = int( 200.0*temp_length/6000.0   ) + 1
                                #window_mean = np.mean(encoded_s[rdx][0,half_window_len:-half_window_len,0,channel_dx])
                                #print('Concate 3 OK')
                                encoded_s[rdx][0,:half_window_len,0,channel_dx] = encoded_s[rdx][0,half_window_len,0,channel_dx]
                                encoded_s[rdx][0,-half_window_len:,0,channel_dx] =  encoded_s[rdx][0,-half_window_len,0,channel_dx]
                                #print('Concate 4 OK')
                                encoded_s[rdx][0,:,0,channel_dx] *= -1.0
                                encoded_s[rdx][0,:,0,channel_dx] -= np.mean(encoded_s[rdx][0,:,0,channel_dx])
                                t_max = np.max(np.abs(encoded_s[rdx][0,:,0,channel_dx]))
                                if t_max < 0.01:
                                    t_max = 1
                                encoded_s[rdx][0,:,0,channel_dx] /= t_max
                                #print('Concate 5 OK')
                                encoded_t[rdx][0,:,0,channel_dx] -= np.max(encoded_t[rdx][0,:,0,channel_dx])
                                encoded_t[rdx][0,:,0,channel_dx] *= -1.0
                                encoded_t[rdx][0,:,0,channel_dx] -= np.mean(encoded_t[rdx][0,:,0,channel_dx])
                                t_max = np.max(np.abs(encoded_t[rdx][0,:,0,channel_dx]))
                                if t_max < 0.01:
                                    t_max = 1
                                encoded_t[rdx][0,:,0,channel_dx] /= t_max
                                #print('Concate 6 OK')
                            siamese_input_list.append(encoded_t[rdx])
                            siamese_input_list.append(encoded_s[rdx])
                    pred_res = siamese_model.predict(siamese_input_list)
                    siamese_s = np.argmax(pred_res[-1][0,:,0,0])
                    pred_amp = pred_res[-1][0,siamese_s,0,0]
                    
                    if pred_amp > 0.4 and siamese_s > 200 and siamese_s < 5800:
                        siamese_s_time = obspy.UTCDateTime(search_key) + siamese_s * 0.01
                        t_update_time  = siamese_s_time - base_time
                        t_diff_time = siamese_s_time - (obspy.UTCDateTime(e_time[-27:]) + sst[0,0] * 60)
                        #r_snr = np.max(np.abs(data_s_in[0,siamese_s-50:siamese_s,0]))/np.max(np.abs(data_s_in[0,siamese_s:siamese_s+100,0]))
                        
                        if np.abs(t_diff_time) < 6:
                            pass
                        else:
                            continue

                        max_pred_amp = pred_amp
                        s_pred_amp = pred_amp
                        siamese_s_time = obspy.UTCDateTime(search_key) + siamese_s * 0.01
                        t_update_time  = siamese_s_time - base_time
                        plot_data_s = data_s_in
                        
                        print(max_pred_amp)
                        print(siamese_s_time)
                        print(obspy.UTCDateTime(e_time[-27:]) + spt[0,0] * 60)


                        eqt_res = EqT_model.predict(plot_data_s)
                        eqt_s = np.argmax(eqt_res[1][0,:,0])
                        pred_amp = eqt_res[1][0,eqt_s,0]
                        if pred_amp > 0.1:
                            pass
                        else:
                            eqt_s = 0.0

                        plt.figure(figsize=(20,16))
                        pdx = 0
                        plt.plot(data_t_in[0,:,0]+pdx*20,color='k')
                        plt.plot(data_t_in[0,:,1]+pdx*20+4,color='k')
                        plt.plot(data_t_in[0,:,2]+pdx*20+8,color='k')
                        plt.plot([spt_t*100,spt_t*100],[pdx*20-10,pdx*20+10],color='g',linewidth=3)

                        pdx = 1
                        plt.plot(plot_data_s[0,:,0]+pdx*20,color='k')
                        plt.plot(plot_data_s[0,:,1]+pdx*20+4,color='k')
                        plt.plot(plot_data_s[0,:,2]+pdx*20+8,color='k')

                        plt.plot([eqt_s,eqt_s],[pdx*20-10,pdx*20+10],color='g',linewidth=3)

                        plt.plot([siamese_s,siamese_s],[pdx*20-7,pdx*20+7],color='r',linewidth=2)
                        plt.title('{}'.format(s_pred_amp),fontsize=30)
                        plt.savefig('t_figs/P_CHECK_{}_{}_{}.png'.format(sta[1][3:],search_sta[1][3:],plot_dx))
                        plot_dx += 1
                        plot_num += 1
                        plt.close()
"""
            if plot_num > 10:
                break
        if plot_num > 10:
            break
    if plot_num > 10:
        break
"""                            
    """
    plt.figure(figsize=(6,6))
    plt.scatter(sta[3],sta[2],color='r')
    for t_sta in search_list:
        plt.scatter(t_sta[3],t_sta[2],color='b')
    plt.xlim([-118.8,-118.0])
    plt.ylim([34.0,34.8])
    plt.show()
    plt.close()
    """
    # open search list csv and hdf5 file
    # for all high P
    #     for all search stations
    #        for all time within range
    #           calcualte P
    #           keep highest Prob
    #           check insert Point
    #           

In [ ]:
def search_update(time_list, updateime, updateamp):
    if len(time_list)

In [ ]:
keep_t = None
for sta_key in s_eqt_dict.keys():
    S_EqT_S_list = list()
    print(sta_key)
    if len(s_eqt_dict[sta_key]['P']) > 0:
        pass
    else:
        continue
    s_times = s_eqt_dict[sta_key]['P']
    s_probs = s_eqt_dict[sta_key]['P_Prob']
    prev_file = '/root/Desktop/Public/SiameseEQTransformer/notebooks/EQT2REAL_PICK/{}.P.txt'.format(sta_key)
    if os.path.exists(prev_file):
        f = open(prev_file,'r')
        for line in f.readlines():
            if len(line) > 3:
                s_times.append(float(line.split(' ')[0]))
                s_probs.append(0.1)
        f.close()
    
    indexs = np.argsort(s_times)
    for idx in range(len(indexs)-1):
        t_id_0 =  indexs[idx]
        t_id_1 = indexs[idx+1]
        if s_times[t_id_1] - s_times[t_id_0] < 3.0:
            if s_probs[t_id_1] >= s_probs[t_id_0]:
                keep_t = s_times[t_id_1]
                keep_prob = s_probs[t_id_1]
            else:
                keep_t = s_times[t_id_0]
                keep_prob = s_probs[t_id_0]
        elif keep_t is None:
            S_EqT_S_list.append('{:.3f} {:.5f} 0.00000000'.format(s_times[t_id_0], s_probs[t_id_0]))
        elif idx == len(indexs)-2:
            S_EqT_S_list.append('{:.3f} {:.5f}  0.00000000'.format(s_times[t_id_0], s_probs[t_id_0]))
            S_EqT_S_list.append('{:.3f} {:.5f}  0.00000000'.format(s_times[t_id_1], s_probs[t_id_1]))
        else:
            S_EqT_S_list.append('{:.3f} {:.5f}  0.00000000'.format(keep_t, keep_prob))
            keep_t = None
    cur_file = '/root/Desktop/Public/SiameseEQTransformer/notebooks/EQT2REAL_PICK_SEQT/{}.P.txt'.format(sta_key)
    f = open(cur_file,'w')
    for S_res in S_EqT_S_list:
        f.write(S_res+'\n')
    f.close()
    #print(S_EqT_S_list)
    #break

In [ ]:
print(S_EqT_S_list)